In [1]:
import time
import json
import datetime as dt
import requests
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.patches as mpatches
import numpy as np
import pwlf

In [16]:
# Selecting the game
game_name = 'hades'

In [17]:
# Getting the game ID
base_url = 'https://www.speedrun.com/api/v1'

response = requests.get(f'{base_url}/games?name={game_name}')
data = response.json()

In [18]:
# Define the status, platform, order, direction, and max results
status = 'verified'
platform = '8gej2n93'  # Platform == PC
orderby = 'date'
direction = 'asc'
max_results = 200  # Maximum results per page

In [19]:
def get_game_id(game_name):
    response = requests.get(f'{base_url}/games?name={game_name}')
    data = response.json()
    if response.status_code == 200:
        return data['data'][0]['id']
    else:
        return None

get_game_id(game_name)

'o1y9okr6'

In [20]:
# Define a function to get the categories for a game
def get_categories(game_id):
    response = requests.get(f'{base_url}/games/{game_id}/categories')
    if response.status_code == 200:
        data = response.json()
        categories = {}
        for category in data['data']:
            categories[category['id']] = category['name']
        return categories
    else:
        print(f'Request failed with status code {response.status_code}.')
        return None

In [21]:
# Define a function to get the runs for a category
def get_runs_for_category(game_id, category_id, category_name):
    extracted_data = []
    page = 0
    while True:
        offset = page * max_results
        response = requests.get(f'{base_url}/runs?game={game_id}&category={category_id}&status={status}&platform={platform}&orderby={orderby}&direction={direction}&max={max_results}&offset={offset}')
        if response.status_code == 200:
            data = response.json()
            if not data['data']:
                break
            for run in data['data']:
                date = run['date']
                submitted = run['submitted']
                verified = run['status']['status']
                primary_t = run['times']['primary_t']
                realtime_t = run['times']['realtime_t']
                extracted_data.append([date, submitted, verified, primary_t, realtime_t, category_name])
            print(f'Page {page + 1} processed for category {category_name}.')
        else:
            print(f'Request failed with status code {response.status_code}. Stopping data collection.')
            print(response.text)
            break
        time.sleep(.6)
        page += 1
    return extracted_data

In [22]:
# Get the game ID
game_id = get_game_id(game_name)

# Get the categories for the game
categories = get_categories(game_id)

# Initialize an empty list to store all the data
all_data = []

# Get the runs for each category
for category_id, category_name in categories.items():
    all_data.extend(get_runs_for_category(game_id, category_id, category_name))

# Convert the list to a DataFrame
df = pd.DataFrame(all_data, columns=['Date', 'Submitted', 'Verified', 'Primary_t', 'Realtime_t', 'Category'])

# Save the DataFrame to a CSV file
df.to_csv('speedrun_data.csv', index=False)

Page 1 processed for category Any Heat.
Page 2 processed for category Any Heat.
Page 3 processed for category Any Heat.
Page 4 processed for category Any Heat.
Page 5 processed for category Any Heat.
Page 6 processed for category Any Heat.
Page 7 processed for category Any Heat.
Page 8 processed for category Any Heat.
Page 9 processed for category Any Heat.
Page 10 processed for category Any Heat.
Page 11 processed for category Any Heat.
Page 12 processed for category Any Heat.
Page 13 processed for category Any Heat.
Page 14 processed for category Any Heat.
Page 15 processed for category Any Heat.
Page 16 processed for category Any Heat.
Page 17 processed for category Any Heat.
Page 18 processed for category Any Heat.
Page 19 processed for category Any Heat.
Page 20 processed for category Any Heat.
Page 21 processed for category Any Heat.
Page 22 processed for category Any Heat.
Page 23 processed for category Any Heat.
Page 24 processed for category Any Heat.
Page 25 processed for cat

In [23]:
# Load the data from the CSV file into a new df
csv_df = pd.read_csv('speedrun_data.csv')
csv_df

Date             Submitted  Verified  Primary_t  Realtime_t  \
0     2020-09-19  2020-09-19T19:56:27Z  verified        490         933   
1     2020-09-19  2020-09-19T19:58:17Z  verified        552        1017   
2     2020-09-22  2020-09-21T15:04:06Z  verified        808        1386   
3     2020-09-22  2020-09-22T10:56:39Z  verified        498         938   
4     2020-09-22  2020-09-22T13:03:25Z  verified        804        1413   
...          ...                   ...       ...        ...         ...   
7207  2020-09-02  2020-09-02T17:50:22Z  verified       6876        6876   
7208  2020-09-05  2020-09-05T14:40:11Z  verified       6630        6630   
7209  2020-09-07  2020-09-06T23:00:37Z  verified       6521        6521   
7210  2020-09-16  2020-09-16T09:59:33Z  verified       6738        6738   
7211  2020-09-18  2020-09-18T19:53:06Z  verified       6950        6950   

                        Category  
0                       Any Heat  
1                       Any Heat  
2                       Any Heat  
3                       Any Heat  
4                       Any Heat  
...                          ...  
7207  All Weapons (Early Access)  
7208  All Weapons (Early Access)  
7209  All Weapons (Early Access)  
7210  All Weapons (Early Access)  
7211  All Weapons (Early Access)  

[7212 rows x 6 columns]

In [24]:
# Filter out rows where 'Date' or 'Submitted' is None
csv_df = csv_df[csv_df['Date'].notnull() & csv_df['Submitted'].notnull()]

# Filter out Realtime that is 0
csv_df = csv_df[csv_df['Realtime_t'] > 0]
csv_df.count()

Date          6448
Submitted     6448
Verified      6448
Primary_t     6448
Realtime_t    6448
Category      6448
dtype: int64

In [25]:
csv_df['Category'].value_counts()

Category
Any Heat                       4419
3 Weapons                       325
OwO                             319
32 Heat                         252
50 Heat                         241
Any Heat (Early Access)         215
40 Heat                         184
Fresh File                      182
All Weapons                     150
All Aspects                      38
Halfspects                       29
All Weapons (Early Access)       26
32 Heat (Early Access)           22
Fresh File (Early Access)        21
FFTC                             20
Highest Heat (Early Access)       5
Name: count, dtype: int64

In [26]:
# Get the category with the most runs
top_category = csv_df['Category'].value_counts().idxmax()

# Filter the DataFrame to only include runs from the top category
csv_df = csv_df[csv_df['Category'] == top_category]
top_category

'Any Heat'

### Preparing data to do linear fit

In [27]:
# Getting the running top score
csv_df['Top_Score'] = csv_df['Realtime_t'].cummin()

# Converting 'Date' is in datetime format
csv_df['Date'] = pd.to_datetime(csv_df['Date'])

# Sort by date
csv_df = csv_df.sort_values('Date')

# Define x (date) and y (top score)
x = csv_df['Date'].map(dt.datetime.toordinal).values
y = csv_df['Top_Score'].values

# Initialize piecewise linear fit with your x and y data
my_pwlf = pwlf.PiecewiseLinFit(x, y)

# Fit the data for two line segments
res = my_pwlf.fit(2)

# Predict for the determined points
xHat = np.linspace(min(x), max(x), num=10000)
yHat = my_pwlf.predict(xHat)

# Convert the ordinal dates back to datetime for plotting
x_dates = [dt.datetime.fromordinal(int(i)) for i in xHat]

In [45]:
# Create a new figure and axes with a larger size
fig, ax = plt.subplots(figsize=(12, 8))

# Plot the piecewise linear fit
ax.plot(x_dates, yHat, linestyle='--', linewidth=3, color='lightgrey', label='Piecewise Linear Fit')

# Plot the original data
ax.plot(csv_df['Date'], csv_df['Top_Score'], linestyle='solid', linewidth=3, color='blue', label='Original Data')

# Get the breakpoints that were calculated during the fit
breakpoints = my_pwlf.fit_breaks

# The elbow is the second breakpoint (the first one is the start of the data)
elbow_x = breakpoints[1]
elbow_y = my_pwlf.predict(elbow_x)

# Convert the ordinal date back to a datetime
elbow_date = dt.datetime.fromordinal(int(elbow_x))

# Define the end point of the line (getting the label closer to the middle of the graph)
end_x = elbow_date + (max(csv_df['Date']) - elbow_date) / 100
end_y = elbow_y + (max(csv_df['Top_Score']) - elbow_y) / 5

# Add the annotation for the elbow with a line going to the middle of the graph
ax.annotate('Efficiency Point', fontweight='bold',
            xy=(elbow_date, elbow_y),
            xytext=(end_x, end_y),
            arrowprops=dict(arrowstyle='-', color='black'))

# Set the title and labels
ax.set_title(f'Top Runs Over Time (Game: "{game_name}", Category: "{top_category}")', fontweight='bold')
ax.set_xlabel('Date', fontweight='bold')
ax.set_ylabel('Top Score (In Seconds)', fontweight='bold')

# Format date
date_format = mdates.DateFormatter('%Y-%m-%d')
ax.xaxis.set_major_formatter(date_format)

# Rotate date labels automatically
fig.autofmt_xdate()

# Create custom legend handles
handle1 = mpatches.Patch(color='blue', label='Original Data')
handle2 = mpatches.Patch(color='lightgrey', label='Piecewise Linear Fit')

# Add the custom legend
ax.legend(handles=[handle1, handle2], loc='upper right')

# Add grid lines
ax.grid(True)

# Show the plot
plt.show()

In [29]:
# Get the index of the elbow in the original data
elbow_index = np.abs(x - elbow_x).argmin()

# Calculate the number of runs to the efficiency point
runs_to_efficiency = elbow_index + 1  # add 1 because indexing starts at 0

# Calculate the percentage of runs complete at efficiency
pct_runs_complete_at_efficiency = (runs_to_efficiency / len(x)) * 100

# Calculate the number of days to the efficiency point
days_to_efficiency = (elbow_date - min(csv_df['Date'])).days

# Calculate the total number of days in the data
total_days = (max(csv_df['Date']) - min(csv_df['Date'])).days

# Calculate the percentage of days to efficiency out of total days
pct_days_to_efficiency = (days_to_efficiency / total_days) * 100

# Print the information
print(f"Number of runs to efficiency point: {runs_to_efficiency}")
print(f"Total runs: {len(x)}")
print(f"Date of efficiency: {elbow_date.strftime('%Y-%m-%d')}")
print(f"Score at efficiency: {elbow_y}")
print(f"Percentage of runs complete at efficiency: {pct_runs_complete_at_efficiency:.2f}%")

# Print the information
print("")
print(f"Number of days to the efficiency point: {days_to_efficiency}")
print(f"Total number of days: {total_days}")
print(f"Percentage of days to efficiency: {pct_days_to_efficiency:.2f}%")

Number of runs to efficiency point: 1466
Total runs: 4419
Date of efficiency: 2021-02-08
Score at efficiency: [770.49087]
Percentage of runs complete at efficiency: 33.17%

Number of days to the efficiency point: 142
Total number of days: 1235
Percentage of days to efficiency: 11.50%
